In [5]:
%%file pg_newmodel2.py
from itertools import count
import gym
import numpy as np
from minirl.agents.pg import PGAgent

from mlopskit import make

model_db = make("cache/feature_store-v1", db_name="lmodel2.db")
model_id = "pg"
env = gym.make("CartPole-v1")
ob_n = env.observation_space.shape[0]
ac_n = env.action_space.n
agent = PGAgent(ob_n, ac_n, model_db=model_db,score_db=model_db,his_db=model_db, model_id=model_id)
render_interval = -1
log_interval = 100


def main():
    """Run REINFORCE algorithm to train on the environment"""
    avg_reward = []
    for i_episode in count(1):
        ep_reward = 0
        obs, _ = env.reset()
        for t in range(100):  # Don't infinite loop while learning

            action = agent.act(obs, model_id)
            next_obs, reward, done, _, _ = env.step(action)
            ep_reward += reward
            #agent.rewards.append(reward)
            agent.policy._add_to_cache_using_rpush("rewards", str(reward), model_id)

            if render_interval != -1 and i_episode % render_interval == 0:
                env.render()

            if done:
                break

            obs = next_obs

        agent.learn(model_id)

        if i_episode % log_interval == 0:
            print("Ave reward: {}".format(sum(avg_reward) / len(avg_reward)))
            avg_reward = []

        else:
            avg_reward.append(ep_reward)


main()


Overwriting pg_newmodel2.py


In [2]:
from itertools import count
import gym
import numpy as np
from minirl.agents.pg import PGAgent

from mlopskit import make

model_db = make("cache/feature_store-v1", db_name="lmodel3.db")
model_id = "pg"
env = gym.make("CartPole-v1")
ob_n = env.observation_space.shape[0]
ac_n = env.action_space.n
agent = PGAgent(ob_n, ac_n, model_db=model_db,score_db=model_db,his_db=model_db, model_id=model_id)
render_interval = -1
log_interval = 100

2023-05-10 08:42:08 [info     ] APIs of mlopskit               model_name=feature_store model_version=1 ops_type=cache
2023-05-10 08:42:08 [info     ] The list of all versions for the current model is [1, 2].
2023-05-10 08:42:08 [warning  ] The version 1 is out of date. You should consider upgrading to version `v2`.
2023-05-10 08:42:08 [info     ] Usage of mlopskit-cache        Params={'db_type': 'rlite/redis/sfdb/diskcache, default:rlite', 'return_type': 'dblink/dbobj, default: dbobj', 'db_name': 'default: rlite_model.cache'}


In [13]:
model_id="pg2"
lu=agent.target_model.cache_local_key("aprobs", model_id)
re=agent.target_model.cache_local_key("rewards", model_id)
#agent.target_model._score_db.delete(re)


In [33]:
l1=agent.target_model._score_db.llen(lu)
l2=agent.target_model._score_db.llen(re)
l1,l2

(643, 642)

In [32]:
agent.target_model._score_db.lrange(re,"0","-1")

['1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',
 '1.0',


In [42]:
%%time
import numpy as np

class LinUCB:
    def __init__(self, alpha, feature_dim):
        self.alpha = alpha
        self.A = np.eye(feature_dim)
        self.b = np.zeros((feature_dim, 1))
        self.theta = np.zeros((feature_dim, 1))

    def update(self, x, r):
        self.A += np.outer(x, x)
        self.b += r * x.reshape(-1, 1)
        self.theta = np.dot(np.linalg.inv(self.A), self.b)

    def get_score(self, x):
        beta = np.sqrt(self.alpha * np.log(np.linalg.det(self.A)) / np.linalg.det(self.A))
        return np.dot(self.theta.T, x) + beta

    def select_arm(self, arms):
        scores = [self.get_score(x) for x in arms]
        print(scores)
        return np.argmax(scores)


# 初始化 LinUCB 模型
linucb = LinUCB(0.1, 4)

# 假设有三个臂，各自对应着一个特征向量（feature_dim=3）
arms = [
    np.array([1.0, 0.0, 0.0,1.0]),
    np.array([0.0, 1.0, 0.0,0.0]),
    np.array([0.0, 0.0, 1.0,1.0])
]

# 选择臂并更新模型（假设选择第一个臂）
chosen_arm = linucb.select_arm(arms)



[array([0.]), array([0.]), array([0.])]
CPU times: user 334 µs, sys: 61 µs, total: 395 µs
Wall time: 390 µs


In [50]:
linucb.update(arms[2], 1.0)

In [54]:
arms[2]

array([0., 0., 1., 1.])

In [53]:
chosen_arm = linucb.select_arm(arms)
chosen_arm

[array([0.50614149]), array([0.67280815]), array([0.83947482])]


2